In [18]:
import wbgapi as wb
wb.data.DataFrame(['NY.GDP.MKTP.CD'], mrv=1)

,NY.GDP.MKTP.CD
economy,
ABW,NaN
AFE,1.236163e+12
AFG,NaN
AFW,7.965862e+11
AGO,8.472296e+10
...,...
XKX,1.043835e+10
YEM,NaN
ZAF,3.777816e+11
